In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from Dataset.WindowDataset import WindowDataset

from Model.AE import Auto
from Model.LSTM import LSTM
from Model.NeuralODE import ODEVAE

from utils import combine_tensors

c:\Users\USER\Desktop\sci\SCI\코드\Model\ODEF.py:169: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xe . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  def forward(self, z0, t=Tensor([0., 1.]), return_whole_sequence=False):


In [3]:
eco = '콜금리'
batch_size = 4
max_apart_size = 70  # 0~60평 아파트만 돌림
real_estate_weighted_average = 0.5
input_size = 5 # LSTM input_size
hidden_size = 16 # LSTM hidden_size

In [4]:
def make_estimate(x):
    vec = np.zeros((max_apart_size))
    vec[int(x[0])] = x[1]
    return vec

In [5]:
def get_date(x):
    date = str(x[1])
    if len(date)==1:
        date = '0'+date
    return str(x[0])+date

In [3]:
df = pd.read_csv('../데이터/merge.csv')

In [19]:
df

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,국고채금리,콜금리
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,200603,10,"59,500",7,1988.0,언주로 103,4.930,3.970
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,200603,29,"60,000",6,1988.0,언주로 103,4.930,3.970
2,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,200604,29,"67,000",9,1988.0,언주로 103,4.950,3.970
3,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,200606,1,"60,000",4,1988.0,언주로 103,4.890,4.170
4,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,200610,20,"72,250",5,1988.0,언주로 103,4.620,4.480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237563,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),59.76,202203,27,"73,000",20,1997.0,동일로 752,2.369,1.244
1237564,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),59.76,202207,20,"74,000",3,1997.0,동일로 752,3.238,1.971
1237565,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),84.03,202207,27,"91,500",12,1997.0,동일로 752,3.238,1.971
1237566,서울특별시 중랑구 중화동,274-51,274.0,51.0,한영(101),84.69,202204,9,"49,900",7,2003.0,동일로144길 74,2.941,1.299


In [21]:
df['시군구'].unique()

array(['서울특별시 강남구 개포동', '서울특별시 강남구 논현동', '서울특별시 강남구 대치동', '서울특별시 강남구 도곡동',
       '서울특별시 강남구 삼성동', '서울특별시 강남구 수서동', '서울특별시 강남구 신사동',
       '서울특별시 강남구 압구정동', '서울특별시 강남구 역삼동', '서울특별시 강남구 일원동',
       '서울특별시 강남구 청담동', '서울특별시 강동구 고덕동', '서울특별시 강동구 길동', '서울특별시 강동구 둔촌동',
       '서울특별시 강동구 명일동', '서울특별시 강동구 상일동', '서울특별시 강동구 성내동', '서울특별시 강동구 암사동',
       '서울특별시 강동구 천호동', '서울특별시 강북구 미아동', '서울특별시 강북구 번동', '서울특별시 강북구 수유동',
       '서울특별시 강북구 우이동', '서울특별시 강서구 가양동', '서울특별시 강서구 공항동',
       '서울특별시 강서구 내발산동', '서울특별시 강서구 등촌동', '서울특별시 강서구 마곡동',
       '서울특별시 강서구 방화동', '서울특별시 강서구 염창동', '서울특별시 강서구 화곡동', '서울특별시 관악구 남현동',
       '서울특별시 관악구 봉천동', '서울특별시 관악구 신림동', '서울특별시 광진구 광장동', '서울특별시 광진구 구의동',
       '서울특별시 광진구 군자동', '서울특별시 광진구 능동', '서울특별시 광진구 자양동', '서울특별시 광진구 중곡동',
       '서울특별시 광진구 화양동', '서울특별시 구로구 가리봉동', '서울특별시 구로구 개봉동',
       '서울특별시 구로구 고척동', '서울특별시 구로구 구로동', '서울특별시 구로구 궁동', '서울특별시 구로구 신도림동',
       '서울특별시 구로구 오류동', '서울특별시 구로구 온수동', '서울특별시 구로구 항동', '서울특별시 금천구 가산동',
       '서울특별시 금천구 독산동', '서울특별시 금천구 시

In [23]:
len(df[df['시군구']=='서울특별시 동작구 본동']['단지명'].unique())

8

In [16]:
df = pd.read_csv('../데이터/merge.csv')
df['시점'] = df[['계약년월','계약일']].apply(get_date, axis=1)
df = df[:100] # /////////////////////////////////
df['시점'] = df['시점'].apply(lambda x : int(x))
df['평수'] = df['전용면적(㎡)'].apply(lambda x : x // 3.3058)
df['가격'] = df['거래금액(만원)'].apply(lambda x : int(x.replace(',','')))
df = df.sort_values(by=['시점'])
df['부동산'] = df[['평수','가격']].apply(make_estimate,axis=1)
df.reset_index(drop=True)
df['경제'] = df[eco]
df = df[['시점','부동산','경제']]
df = df.reset_index(drop=True)

NameError: name 'get_date' is not defined

In [7]:
df

,시점,부동산,경제
0,20060103,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.74
1,20060103,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.74
2,20060103,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.74
3,20060103,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.74
4,20060104,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.74
...,...,...,...
95,20061030,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.48
96,20061031,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.48
97,20061103,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.47
98,20061103,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.47


In [8]:
len_df = len(df)
train_len = int(len_df *0.6)
val_len = int(len_df * 0.3)

batch_size = 4

train_dataset = WindowDataset(df[:train_len])
train_loader = DataLoader(train_dataset, batch_size=batch_size)

val_dataset = WindowDataset(df[train_len:train_len+val_len])
val_loader = DataLoader(val_dataset, batch_size=batch_size)

test_dataset = WindowDataset(df[train_len+val_len:])
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [9]:
# x1 : 시점/경제 데이터, x2 : 부동산 데이터
# y1 : 시점/경제 라벨링, y2 : 부동산 라벨링

for x1,x2,y1,y2 in train_loader:
    print(x1.shape)
    print(x2.shape)
    print(y1.shape)
    print(y2.shape)
    break

torch.Size([4, 5, 2])
torch.Size([4, 5, 70])
torch.Size([4, 2])
torch.Size([4, 70])


In [10]:
# 모델 입출력 크기

input_size = 256
window_size = 5
hidden_size = 512
latent_size = 512

In [11]:
######### 모델
Model = ODEVAE(input_size,hidden_size,latent_size)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = Model.to(device)

print('작동하는지 실험')
basic_data = torch.rand((window_size,1,input_size))
t = torch.rand((window_size,1,1))
model(basic_data,t)

작동하는지 실험


(tensor([[[-0.0344]],
 
         [[-0.0295]],
 
         [[-0.0336]],
 
         [[-0.0326]],
 
         [[-0.0414]]], grad_fn=<ViewBackward0>),
 tensor([[ 2.6171e+00, -1.6073e+00,  1.1797e-01,  1.4978e+00,  5.0095e-01,
          -2.8180e-01, -1.9412e+00, -2.6127e+00,  6.0840e-01, -1.1363e+00,
           9.1898e-01, -1.4987e-01, -1.4771e-02, -9.0110e-01,  6.9646e-01,
          -1.8452e-01, -2.0950e-01, -4.7863e-05, -1.1433e+00, -1.1165e-01,
          -1.1798e+00,  1.0647e+00,  1.5398e+00, -2.1322e-01,  1.7331e+00,
          -1.0525e+00, -6.4346e-01, -3.9084e-01, -1.6705e+00, -1.3810e+00,
          -1.3874e-01, -4.9260e-01, -2.1583e-01,  5.1785e-01, -7.0057e-01,
          -8.7429e-03, -3.1878e+00, -2.1714e+00, -1.6796e+00, -4.0784e-01,
           1.4370e-01,  3.1235e-01, -4.7366e-01,  1.4165e+00, -1.7835e-02,
          -4.3294e-01,  1.2682e+00,  3.7691e-01,  2.4093e-01, -7.8649e-01,
          -7.5837e-01,  1.3472e+00,  3.1952e-01, -1.6144e+00,  6.5721e-01,
          -1.4676e+00, -1.7814

In [12]:
#############################
# 데이터 & 모델에 device 붙임!!!
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

# model = Model.to(device)

# print('작동하는지 실험')
# basic_data = torch.rand((1,input_size))
# model(basic_data)

criterion = nn.MSELoss()

lr = 1e-3
num_epochs = 10
noise_std = 0.02
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

cpu is available


In [13]:
economy_embedding_model = LSTM(input_size,hidden_size).to(device)
economy_embedding_model.load_state_dict(torch.load('Checkpoint/economic_best_model.pth'))

real_estate_embedding_model = Auto(max_apart_size).to(device)
real_estate_embedding_model.load_state_dict(torch.load('Checkpoint/embedding_best_model.pth'))

RuntimeError: Error(s) in loading state_dict for Auto:
	Unexpected key(s) in state_dict: "encoder.4.weight", "encoder.4.bias", "decoder.4.weight", "decoder.4.bias". 
	size mismatch for encoder.0.weight: copying a param with shape torch.Size([64, 7]) from checkpoint, the shape in current model is torch.Size([32, 70]).
	size mismatch for encoder.0.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for encoder.2.weight: copying a param with shape torch.Size([64, 64]) from checkpoint, the shape in current model is torch.Size([16, 32]).
	size mismatch for encoder.2.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([16]).
	size mismatch for decoder.0.weight: copying a param with shape torch.Size([64, 256]) from checkpoint, the shape in current model is torch.Size([32, 16]).
	size mismatch for decoder.0.bias: copying a param with shape torch.Size([64]) from 

In [ ]:
for epoch in range(num_epochs):
    print('epoch : {}'.format(epoch))
    running_loss = 0.0
    num = 0
    print("train")
    model.train()
    for x1, x2, y1, y2 in train_loader:
        x1, x2 = x1.to(device), x2.to(device) # x1: [시점, 경제], x2: 부동산
        y1, y2 = y1.to(device), y2.to(device)
        
        time_data = x1[:,:,0]
        eco_data = x1[:,:,1]
        
        # 경제 데이터
        eco_emb = economy_embedding_model(eco_data)
        # 부동산 데이터
        real_est_emb = real_estate_embedding_model(x2)
        
        final_emb = combine_tensors(real_est_emb, eco_emb, real_estate_weighted_average, 'sum')
        ##########################
        # 모델에 x[0], final_emb 가지고 ODE 학습하기!!!!
        
        optimizer.zero_grad()

        x_p, z, z_mean, z_log_var = model(final_emb, time_data)
        kl_loss = -0.5 * torch.sum(1 + z_log_var - z_mean**2 - torch.exp(z_log_var), -1)
        loss = 0.5 * ((final_emb-x_p)**2).sum(-1).sum(0) / noise_std**2 + kl_loss
        loss = torch.mean(loss)
       
        loss.backward()
        optimizer.step()